In [1]:
!pip install sdv

  Using cached sdv-1.22.1-py3-none-any.whl.metadata (14 kB)
  Using cached boto3-1.38.36-py3-none-any.whl.metadata (6.6 kB)
  Using cached botocore-1.38.36-py3-none-any.whl.metadata (5.7 kB)
  Using cached copulas-0.12.3-py3-none-any.whl.metadata (9.5 kB)
  Using cached ctgan-0.11.0-py3-none-any.whl.metadata (10 kB)
  Using cached deepecho-0.7.0-py3-none-any.whl.metadata (10 kB)
  Using cached rdt-1.17.0-py3-none-any.whl.metadata (10 kB)
  Using cached sdmetrics-0.21.0-py3-none-any.whl.metadata (9.4 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.13.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached faker-37.4.0-py3-none-any.whl.metadata (15 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 k

In [2]:
!unrar x  data.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from data.rar

Creating    data                                                      OK
Extracting  data/california_housing.csv                                   24%  OK 
Extracting  data/insurance.csv                                            31%  OK 
Extracting  data/my_classification.csv                                    55%  OK 
Extracting  data/my_regression.csv                                        80%  OK 
Extracting  data/titanic.csv                                              89%  OK 
Extracting  data/two_moons.csv                                            99%  OK 
All OK


In [3]:
import pandas as pd
import numpy as np
import torch
import random
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from hyperopt import fmin, rand, hp, Trials, STATUS_OK
from hyperopt import space_eval
from sdv.single_table import CTGANSynthesizer as CTGAN
from sdv.metadata import SingleTableMetadata
from sklearn.metrics import accuracy_score, f1_score
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import f1_score, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [4]:
RANDOM_SEED = 42

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)

In [12]:
    dir_datasets = 'data/'

    # Загрузка реальных датасетов
    real_data_1 = pd.read_csv(dir_datasets+'insurance.csv')
    real_data_2 = pd.read_csv(dir_datasets+'my_classification.csv')
    real_data_3 = pd.read_csv(dir_datasets+'my_regression.csv')
    real_data_4 = pd.read_csv(dir_datasets+'two_moons.csv')
    real_data_5 = pd.read_csv(dir_datasets+'california_housing.csv')
    real_data_6 = pd.read_csv(dir_datasets+'titanic.csv')

    # Словарь датасетов для удобства
    datasets = {
        'titanic': {
                        "data": real_data_6,
                        "task": "classification",
                        "target": "Survived",
                        "num_columns": ["Age", "Fare", "SibSp", "Parch"],
                        "cat_columns": ["Pclass", "Sex", "Embarked"]
                    },

    }

    for name, data in datasets.items():
        print(f" Информация о датасете {name}:")
        print(f" Количество строк: {data['data'].shape[0]}")
        print(f" Количество колонок: {data['data'].shape[1]}")
        print(f" Колонки: {list(data['data'].columns)}")
        print(f" Задача: {data['task']}")
        print(f" Целевая переменная: {data['target']}")
        print(f" Числовые признаки: {data['num_columns']}")
        print(f" Категориальные признаки: {data['cat_columns']}")
        print()

 Информация о датасете titanic:
 Количество строк: 712
 Количество колонок: 12
 Колонки: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
 Задача: classification
 Целевая переменная: Survived
 Числовые признаки: ['Age', 'Fare', 'SibSp', 'Parch']
 Категориальные признаки: ['Pclass', 'Sex', 'Embarked']



In [13]:
def process_data(dataset, num_columns, cat_columns, transformation_num_type='None', transformation_cat_type='None'):

    df_processed = dataset.copy()

    # Обработка числовых признаков
    if transformation_num_type == 'CDF':
        # CDF трансформация: преобразует значения в их эмпирическую функцию распределения
        for col in num_columns:
            # Правильная формула для эмпирической CDF
            df_processed[col] = (df_processed[col].rank(method='average') - 0.5) / len(df_processed)

    elif transformation_num_type == 'PLE_CDF':
        # PLE_CDF (Probability Logit Envelope CDF) - более сложная трансформация
        for col in num_columns:
            # Сначала применяем CDF
            cdf_values = (df_processed[col].rank(method='average') - 0.5) / len(df_processed)

            # Затем применяем logit трансформацию с небольшим сглаживанием
            # Избегаем 0 и 1 для предотвращения бесконечности в logit
            epsilon = 1e-6
            cdf_values = np.clip(cdf_values, epsilon, 1 - epsilon)

            # Логит трансформация: ln(p/(1-p))
            df_processed[col] = np.log(cdf_values / (1 - cdf_values))

    # Обработка категориальных признаков
    if transformation_cat_type == 'OHE':
        # One Hot Encoding для категориальных признаков
        for col in cat_columns:
            # Создаем dummy переменные с префиксом имени колонки
            dummy_df = pd.get_dummies(df_processed[col], prefix=col, dtype=int)

            # Удаляем исходную категориальную колонку
            df_processed = df_processed.drop(columns=[col])

            # Добавляем новые dummy колонки
            df_processed = pd.concat([df_processed, dummy_df], axis=1)

    return df_processed

In [14]:
ctgan_space = {
    # Learning rates
    'discriminator_lr': hp.qloguniform('discriminator_lr',
                                                 np.log(4e-4), np.log(2.1e-3), 5e-5),
    'generator_lr': hp.qloguniform('generator_lr',
                                             np.log(5e-5), np.log(5e-3), 5e-5),

    # Batch size
    'batch_size': hp.choice('batch_size', [100, 500, 1000]),

    # Embedding dimensions
    'embedding_dim': hp.choice('embedding_dim', [32, 128]),

    # Network architecture
    'generator_dim': hp.choice('generator_dim', [[128, 128, 128], [128, 128, 128, 128]]),
    'discriminator_dim': hp.choice('discriminator_dim', [[256, 256], [256, 256, 256]]),

    # Decay parameters
    'generator_decay': hp.qloguniform('generator_decay',
                                     np.log(1e-6), np.log(6.4e-6), 1e-7),
    'discriminator_decay': hp.qloguniform('discriminator_decay',
                                         np.log(1e-6), np.log(8e-6), 1e-6),

    # Frequency
    'log_frequency': hp.choice('log_frequency', [False, True]),
    'transformation_num_type': hp.choice('transformation_num_type', ['CDF', 'PLE_CDF']),
    'transformation_cat_type': hp.choice('transformation_cat_type', ['OHE'])

}

In [15]:
def evaluate_ml_efficacy(real_train, synthetic_train, real_test, target_column, num_columns, cat_columns, task_type='classification'):
    """
    Вычисляет ML-Efficacy метрику

    Args:
        real_train: реальные тренировочные данные
        synthetic_train: синтетические тренировочные данные
        real_test: реальные тестовые данные
        target_column: название целевой колонки
        task_type: 'classification' или 'regression'

    Returns:
        ml_efficacy_score: отношение качества модели на синтетических данных к качеству на реальных
    """

    # Подготовка данных
    real_train_copy = real_train.copy()

    # Обработка синтетических данных (если это объект с методом dataframe)
    if hasattr(synthetic_train, "dataframe"):
        synthetic_train = synthetic_train.dataframe()
    synthetic_train_copy = synthetic_train.copy()

    real_test_copy = real_test.copy()

    # Определение категориальных колонок (исключая целевую)
    feature_columns = num_columns
    categorical_columns = cat_columns

    # Label Encoding для категориальных признаков
    label_encoders = {}
    for col in categorical_columns:
        le = LabelEncoder()
        # Обучаем на реальных данных
        combined_values = pd.concat([real_train_copy[col], real_test_copy[col]]).astype(str)
        le.fit(combined_values)
        label_encoders[col] = le

        # Применяем к реальным данным
        real_train_copy[col] = le.transform(real_train_copy[col].astype(str))
        real_test_copy[col] = le.transform(real_test_copy[col].astype(str))

        # Применяем к синтетическим данным, обрабатываем неизвестные значения
        synthetic_values = synthetic_train_copy[col].astype(str)
        known_values = set(le.classes_)
        synthetic_values = synthetic_values.apply(lambda x: x if x in known_values else le.classes_[0])
        synthetic_train_copy[col] = le.transform(synthetic_values)

    # Разделение на признаки и целевую переменную
    X_real_train = real_train_copy.drop(columns=[target_column])
    y_real_train = real_train_copy[target_column]

    X_synthetic_train = synthetic_train_copy.drop(columns=[target_column])
    y_synthetic_train = synthetic_train_copy[target_column]

    X_real_test = real_test_copy.drop(columns=[target_column])
    y_real_test = real_test_copy[target_column]

    # Выбор модели в зависимости от типа задачи
    if task_type == 'classification':
        model_real = XGBClassifier(random_state=42, eval_metric='logloss')
        model_synthetic = XGBClassifier(random_state=42, eval_metric='logloss')
        score_func = f1_score
        score_kwargs = {'average': 'weighted'} if len(np.unique(y_real_train)) > 2 else {'average': 'binary'}
    else:  # regression
        model_real = XGBRegressor(random_state=42)
        model_synthetic = XGBRegressor(random_state=42)
        score_func = r2_score
        score_kwargs = {}

    try:
        # Обучение модели на реальных данных
        model_real.fit(X_real_train, y_real_train)
        y_pred_real = model_real.predict(X_real_test)
        score_real = score_func(y_real_test, y_pred_real, **score_kwargs)

        # Обучение модели на синтетических данных
        model_synthetic.fit(X_synthetic_train, y_synthetic_train)
        y_pred_synthetic = model_synthetic.predict(X_real_test)
        score_synthetic = score_func(y_real_test, y_pred_synthetic, **score_kwargs)

        # Вычисление ML-Efficacy
        if score_real > 0:
            ml_efficacy = score_synthetic / score_real
        else:
            ml_efficacy = 0.0

        return ml_efficacy

    except Exception as e:
        print(f"Ошибка при вычислении ML-Efficacy: {e}")
        return 0.0

In [16]:
from sklearn.model_selection import KFold

def generate_and_evaluate_ml_efficacy(params, dataset_info, n_splits=3):
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=UserWarning)

    # Извлекаем информацию о задаче и целевой переменной из dataset_info
    task_type = dataset_info['task']
    target_column = dataset_info['target']

    columns_dataset = dataset_info['num_columns'] + dataset_info['cat_columns'] + [dataset_info['target']]
    data = process_data(dataset_info['data'][columns_dataset], dataset_info['num_columns'], dataset_info['cat_columns'],
                        'None', 'None')

    kf = KFold(n_splits=n_splits, shuffle=False)
    ml_efficacy_scores = []

    for train_index, test_index in kf.split(data):
        train_data = data.iloc[train_index].reset_index(drop=True)
        test_data = data.iloc[test_index].reset_index(drop=True)

        metadata = SingleTableMetadata()
        metadata.detect_from_dataframe(data=train_data)

        # Создание и обучение CTGAN с заданными параметрами
        ctgan = CTGAN(
            metadata=metadata,
            discriminator_lr=params['discriminator_lr'],
            generator_lr=params['generator_lr'],
            batch_size=params['batch_size'],
            embedding_dim=params['embedding_dim'],
            generator_dim=params['generator_dim'],
            generator_decay=params['generator_decay'],
            discriminator_decay=params['discriminator_decay'],
            log_frequency=params['log_frequency'],
        )

        ctgan.fit(train_data)

        synthetic_data = ctgan.sample(len(test_data))

        num_columns = dataset_info['num_columns'].copy()
        cat_columns = dataset_info['cat_columns'].copy()
        if dataset_info["task"] == 'classification':
            cat_columns.append(dataset_info['target'])

        # Вычисляем ML-Efficacy
        score = evaluate_ml_efficacy(
            real_train=train_data,
            synthetic_train=synthetic_data,
            real_test=test_data,
            target_column=target_column,
            num_columns = num_columns,
            cat_columns = cat_columns,
            task_type=task_type
        )
        ml_efficacy_scores.append(score)

    return np.mean(ml_efficacy_scores)

In [17]:
def optimize_dataset(dataset_name, dataset_info, max_evals=30):
    print(f"Оптимизация для датасета: {dataset_name}")

    # Создаем объект для хранения истории поиска
    trials = Trials()

    # Определяем целевую функцию для оптимизации
    def objective(params):
        print("="*50)
        ml_efficacy_score = generate_and_evaluate_ml_efficacy(params, dataset_info)

        print(f"ML-Efficacy: {ml_efficacy_score}")
        print(params)
        print("="*50)

        # Для ML-Efficacy мы хотим максимизировать метрику, поэтому возвращаем отрицательное значение
        return {
            'loss': -ml_efficacy_score,  # отрицательное для максимизации
            'status': STATUS_OK,
            'ml_efficacy_score': ml_efficacy_score
        }

    # Запускаем оптимизацию с помощью TPE алгоритма
    rng = np.random.default_rng(42)  # Используем фиксированный seed
    best = fmin(
        fn=objective,
        space=ctgan_space,
        algo=rand.suggest,
        max_evals=max_evals,
        trials=trials,
        rstate=rng
    )

    # Автоматическое декодирование параметров
    best_params = space_eval(ctgan_space, best)

    # Находим лучший результат
    best_trial = min(trials.trials, key=lambda x: x['result']['loss'])
    best_ml_efficacy = best_trial['result'].get('ml_efficacy_score', None)

    results = {
        'best_params': best_params,
        'best_ml_efficacy': best_ml_efficacy,
        'ml_efficacy_diff_from_optimal': -best_trial['result']['loss']  # убираем минус для читаемости
    }

    print(f"Лучший ML-Efficacy для {dataset_name}: {best_ml_efficacy}")
    print(f"Лучшие параметры: {best_params}")
    print('-' * 50)

    return results

In [18]:
# Словарь для хранения результатов
all_results = {}

# Указываем количество итераций для каждого датасета
max_evals = 50

# Запускаем оптимизацию для каждого датасета
for dataset_name in datasets:
    data_info = datasets[dataset_name]
    all_results[dataset_name] = optimize_dataset(dataset_name, data_info, max_evals)

Оптимизация для датасета: titanic
ML-Efficacy: 0.7569069095695212
{'batch_size': 100, 'discriminator_decay': 4e-06, 'discriminator_dim': (256, 256, 256), 'discriminator_lr': 0.0014500000000000001, 'embedding_dim': 128, 'generator_decay': 3.3999999999999996e-06, 'generator_dim': (128, 128, 128), 'generator_lr': 0.0008500000000000001, 'log_frequency': True, 'transformation_cat_type': 'OHE', 'transformation_num_type': 'CDF'}
ML-Efficacy: 0.7903936933958507
{'batch_size': 100, 'discriminator_decay': 6e-06, 'discriminator_dim': (256, 256, 256), 'discriminator_lr': 0.001, 'embedding_dim': 128, 'generator_decay': 1.1e-06, 'generator_dim': (128, 128, 128, 128), 'generator_lr': 0.0008, 'log_frequency': False, 'transformation_cat_type': 'OHE', 'transformation_num_type': 'PLE_CDF'}
ML-Efficacy: 0.5972506142651288
{'batch_size': 1000, 'discriminator_decay': 6e-06, 'discriminator_dim': (256, 256, 256), 'discriminator_lr': 0.0010500000000000002, 'embedding_dim': 128, 'generator_decay': 1.29999999999

In [19]:
for dataset_name in datasets:
    print(dataset_name, all_results[dataset_name]['best_ml_efficacy'])

titanic 0.8056613580640383
